In [23]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,2 days 7 hours 36 mins
H2O cluster version:,3.16.0.4
H2O cluster version age:,1 month and 5 days
H2O cluster name:,root
H2O cluster total nodes:,1
H2O cluster free memory:,3.530 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


In [45]:
docker_data_path = "/volume_data_files/very_clean_jokes.csv"
jokes = h2o.import_file(docker_data_path, col_types=['int','string'])

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [46]:
jokes.tail(2)

is_dad_joke,dad_joke
0,What would it be like to have world with out women?? It would be a real pain in the ass
0,"Old sex life. An elderly woman goes to the doctor and asks his help to revive her Husband's sex drive. 'What about trying Viagra?' asks the doctor. Not a chance' says Mrs. Murphy. """"He won't even take an aspirin for a headache."""" 'No problem,' replies the doctor. 'Drop it into his coffee,he won't even taste it. Try it and come back in a week to let me know how you got on.' A week later Mrs. Murphy returns to the doctor and he inquires as to how things went. 'Oh it was terrible, just terrible doctor.' What happened?' asks the doctor. 'Well I did as you advised and slipped it in his coffee. The effect was immediate. He jumped straight up, swept the cutlery off the table, at the same time ripping my clothes off and then proceeded to make wild passionate love to me on the tabletop. It was terrible! 'What was terrible?' said the doctor, 'Was the sex not good?' 'Oh no doctor, the sex was the best I've had in 25 years... but I'll never be able to show my face in McDonald's again"


In [47]:
jokes['is_dad_joke'].table()

is_dad_joke,Count
0,999
1,3796


In [48]:
import pandas as pd
import os

# Use local data file or download from GitHub
docker_data_path = "/home/h2o/data/nlp/stopwords.csv"
if os.path.isfile(docker_data_path):
  data_path = docker_data_path
else:
  data_path = "https://raw.githubusercontent.com/h2oai/h2o-tutorials/master/h2o-world-2017/nlp/stopwords.csv"

STOP_WORDS = pd.read_csv(data_path, header=0)
STOP_WORDS = list(STOP_WORDS['STOP_WORD'])

In [50]:
def tokenize(sentences, stop_word = STOP_WORDS):
    tokenized = sentences.tokenize("\\W+")
    tokenized_lower = tokenized.tolower()
    tokenized_filtered = tokenized_lower[(tokenized_lower.nchar() >= 2) | (tokenized_lower.isna()),:]
    tokenized_words = tokenized_filtered[tokenized_filtered.grep("[0-9]",invert=True,output_logical=True),:]
    tokenized_words = tokenized_words[(tokenized_words.isna()) | (~ tokenized_words.isin(STOP_WORDS)),:]
    return tokenized_words

In [51]:
words = tokenize(jokes["dad_joke"], STOP_WORDS)

In [52]:
words.head()

C1
fish
say
swam
wall
damn
""
birds
fly
south
winter


In [53]:
# Train Word2Vec Model
from h2o.estimators.word2vec import H2OWord2vecEstimator

w2v_model = H2OWord2vecEstimator(vec_size = 100, model_id = "w2v.hex")
w2v_model.train(training_frame=words)

word2vec Model Build progress: |██████████████████████████████████████████| 100%


In [57]:
w2v_model.find_synonyms("nurse", count = 5)

OrderedDict([('invisible', 0.7891305685043335),
             ('patient', 0.6975910663604736),
             ('hip', 0.6818426251411438),
             ('coolest', 0.6720466017723083),
             ('sighs', 0.6598649621009827)])

In [58]:
review_vecs = w2v_model.transform(words, aggregate_method = "AVERAGE")

In [60]:
review_vecs


C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
-0.164148,0.176695,-0.0376753,-0.322354,-0.225436,-0.0612733,0.0348412,-0.150792,0.121014,0.0505698,0.089769,-0.199452,-0.0904864,-0.157586,0.16909,-0.00563268,-0.131225,-0.116459,-0.186428,-0.111921,-0.290799,0.126358,-0.0904237,0.252726,-0.0164348,0.136646,-0.0885142,-0.365798,0.025545,-0.249629,0.365286,-0.102065,-0.188902,0.08732,-0.202859,0.113402,-0.0159578,-0.169287,-0.166655,0.287639,0.235626,0.000125486,0.04075,0.184574,0.0343147,0.0453349,0.224679,-0.179168,-0.197052,0.170124,0.121012,-0.332459,0.154378,-0.0579608,-0.264666,-0.0309094,0.126308,0.0541831,-0.401629,-0.0258865,-0.102126,-0.0640541,0.160266,-0.15915,-0.145938,-0.334105,-0.108451,0.0407329,0.0348034,0.382574,-0.0409882,-0.0479708,-0.0429055,0.0366991,-0.0105829,-0.339749,0.157117,0.176853,0.0542374,-0.23875,0.158308,0.215192,-0.176469,-0.0288284,-0.26739,0.0116702,0.354108,-0.422806,-0.224821,-0.00288463,0.0335843,-0.224617,0.0131119,-0.0528691,-0.0818662,0.098528,-0.205722,0.0882565,-0.286009,0.00583597
0.163545,0.458362,-0.155971,-0.0253381,-0.105555,0.12778,0.0437156,0.0816394,-0.246265,-0.0990371,0.2344,0.053279,-0.190363,0.00981104,-0.0865712,-0.235138,0.191495,0.0562362,-0.236155,-0.098442,0.200792,-0.00434632,0.0471326,0.148141,0.151041,-0.00693277,-0.0885744,-0.0486047,-0.197384,-0.0902863,-0.248408,0.146592,0.0416159,0.084761,0.0399147,0.409352,0.407292,-0.184929,0.0094233,0.222626,0.238974,0.0209618,-0.160799,-0.196311,-0.0844792,-0.196302,0.0926721,0.0384261,0.0111173,0.125215,-0.130925,-0.103492,0.010229,0.226808,0.201667,-0.298628,-0.101916,0.193203,-0.00373814,-0.0683055,-0.0735013,-0.0627803,-0.319175,-0.317237,-0.0491757,0.124252,-0.115357,-0.232173,-0.113483,0.309816,0.0629495,-0.270774,-0.0469463,-0.0806987,-0.114745,0.0743491,-0.21239,0.102368,0.241968,-0.141948,-0.16139,0.115162,-0.43829,-0.170253,-0.147222,-0.0394893,-0.160344,-0.254703,-0.120758,-0.0631472,-0.0731593,0.0109954,-0.0541612,0.243896,-0.254191,0.235261,-0.18031,-0.111913,-0.593839,-0.219675
-0.106225,0.133158,-0.177425,0.121794,0.269968,-0.258321,0.0999803,-0.624233,0.0254023,-0.167209,-0.257123,-0.191527,-0.181489,0.373565,-0.0131161,-0.0726535,-0.0670855,-0.126343,-0.119768,0.0901216,-0.218052,-0.18025,-0.15294,-0.00630674,0.0816288,-0.0622691,0.108736,-0.252445,0.049124,-0.198836,-0.0640196,0.0461506,0.040217,-0.119745,-0.284689,0.00319259,0.286784,-0.164437,-0.150161,-0.289653,0.142363,0.207271,-0.22764,-0.0914545,-0.256766,-0.16937,0.306239,-0.00758743,-0.44422,0.0438933,0.170846,-0.274112,-0.0607665,-0.280125,0.373651,-0.726014,-0.0340954,0.097993,0.0887939,-0.00367886,-0.0863853,0.0405809,-0.0276034,-0.271072,-0.442069,-0.271585,-0.281401,0.3058,-0.165068,0.139148,-0.0485183,-0.286203,0.213913,-0.368647,-0.0263262,-0.117094,-0.089832,-0.0698997,-0.171992,-0.0351965,0.143697,0.214405,-0.13681,0.300929,-0.285458,0.180971,0.141542,0.0755244,0.388076,-0.0180776,-0.175615,0.0830056,0.233,0.297035,-0.213546,0.284216,-0.196265,0.00403209,-0.218741,0.23769
-0.267708,-0.166831,-0.0223698,-0.120573,-0.00298901,0.0132561,-0.049119,-0.225055,0.153969,0.212055,0.0286833,-0.377117,-0.115373,-0.0141007,0.16465,0.0643532,0.0731689,-0.0514693,0.0143636,0.140507,-0.447962,0.139853,0.0395575,0.0684695,-0.0350805,-0.0555583,-0.267917,-0.11322,-0.102601,-0.108961,-0.194573,0.157354,0.0345626,0.332272,0.0551752,-0.0731744,0.00478324,0.27418,0.167127,0.162382,0.151913,-0.0322361,0.017764,0.0270358,0.0731943,-0.253901,0.0946305,-0.0606948,0.113974,0.124358,0.172444,-0.137007,-0.250482,-0.210969,0.0189257,-0.183414,0.0296906,-0.106453,-0.249105,-0.0849207,0.0449182,-0.162364,0.10838,-0.296897,-0.440557,-

In [67]:
ext_reviews = jokes.cbind(review_vecs)
ext_reviews.tail()

is_dad_joke,dad_joke,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26,C27,C28,C29,C30,C31,C32,C33,C34,C35,C36,C37,C38,C39,C40,C41,C42,C43,C44,C45,C46,C47,C48,C49,C50,C51,C52,C53,C54,C55,C56,C57,C58,C59,C60,C61,C62,C63,C64,C65,C66,C67,C68,C69,C70,C71,C72,C73,C74,C75,C76,C77,C78,C79,C80,C81,C82,C83,C84,C85,C86,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96,C97,C98,C99,C100
0,"A woman walks into a dry cleaner... and says """"I've got another dress for you"""" The man behind the counter, whose a little hard of hearing, reply """"come again?"""" The woman responds with """"No this time its mus",0.0184974,-0.123192,0.00452156,0.0206166,-0.0241109,-0.238644,-0.0833693,-0.236311,0.0525549,-0.0598623,0.0884177,0.0584373,-0.0496448,-0.00198575,0.133713,-0.0539329,0.003111,0.061544,-0.0116063,0.0976455,-0.0532309,-0.0885592,0.04641,-0.0148731,-0.102459,0.119683,-0.0969967,0.0709647,0.130245,0.05601,0.0524674,-0.020395,0.102683,0.0328782,-0.0218598,-0.0368207,-0.0402088,-0.120149,0.0239493,0.151769,0.0981955,-0.171936,0.0401704,-0.123934,-0.0523777,-0.158069,0.157407,0.0534071,-0.0224234,-0.0160882,0.121195,-0.0893226,-0.0259718,-0.0804814,0.156495,-0.0589779,0.0428274,0.147787,-0.000255881,-0.0229667,-0.1275,0.000668445,-0.0702201,-0.0438246,0.094738,-0.151108,-0.0644893,-0.0223067,-0.0331342,-0.0879664,-0.0939484,-0.129102,0.0480567,-0.0148438,0.105688,0.0331661,-0.0421125,0.0360889,0.0813952,-0.254429,0.0424211,-0.0966758,-0.168446,-0.0341546,-0.0464564,0.0626277,-0.108395,-0.208146,-0.126594,-0.119009,0.00964537,-0.162848,-0.0931006,0.105193,-0.13189,0.104888,0.00995113,0.0436997,-0.107606,0.00932241
0,"2 Guys in a Bar 2 guys in a bar. John: """"I have sex with my wife once a month"""" Jack: """"We do it twice a week"""" John: """"But Jack, you're not even married!"""" Jack: """"Oh, I thought we were talking about you",0.0424964,-0.181451,-0.0119709,0.0253887,-0.143112,-0.193139,-0.033069,-0.252105,0.0522089,0.0181782,-0.0228087,-0.0302313,-0.0848376,-0.00591299,0.109793,0.00413202,0.0654173,0.0649728,-0.0328553,0.105737,-0.0303708,0.0253002,-0.0314987,-0.0111447,-0.0455228,0.0150723,-0.134802,0.0398907,0.133311,0.0338944,0.0648472,0.045035,0.0686413,0.0824648,0.0389045,0.00990517,0.0110449,-0.00788786,0.0206184,0.160586,0.0181816,-0.134408,0.00187266,-0.1088,0.0329046,-0.0993511,0.156352,0.152856,7.84609e-05,-0.0215429,0.115734,-0.102016,-0.0163079,-0.109748,0.195018,-0.096638,0.114794,0.158476,-0.0252376,0.0450864,-0.12017,0.0373412,-0.0882368,-0.226502,0.118733,-0.175397,-0.0400125,0.0172252,-0.173109,0.0195659,-0.0715622,-0.107996,0.107911,-0.0156382,-0.0541659,0.0713073,-0.0341884,-0.102857,0.0392727,-0.137801,0.0220682,-0.00469767,-0.13834,0.00701349,0.0488041,0.0457093,-0.145161,-0.142836,-0.0390791,-0.0409434,0.0417182,-0.174703,-0.17602,0.154539,-0.176223,0.172012,-0.091699,0.00183727,-0.0940386,0.000342786
0,"To all new iPad owners. When you masturbate in front of your new device, make sure to put some porn on its screen to avoid embarrassment.",-0.0712686,-0.208309,-0.0419731,0.0934983,-0.0513367,-0.186157,-0.0323605,-0.0198508,0.0486718,-0.0843393,-0.0548129,-0.100125,-0.121938,-0.0836084,0.0893008,-0.021305,-0.0499233,0.0669963,0.0963745,0.0839142,-0.0840473,0.0320954,0.176293,-0.211811,-0.0384895,0.14274,-0.0903161,0.055297,-0.0336762,-0.0203859,0.159088,0.100482,0.286916,0.0616306,0.0974116,-0.0239264,0.0347988,-0.0327657,0.0323693,0.230015,0.105994,-0.0478504,0.0603314,0.138779,0.00817079,-0.165203,0.190148,0.095225,0.00407428,-0.0722893,0.0985165,-0.0155517,-0.0748637,-0.19644,0.106741,-0.0329535,-0.0531186,0.260029,-0.104146,0.0144429,0.0207957,-0.0610924,0.0469189,-0.00537564,0.208792,-0.223341,0.0827998,-0.0127643,0.0978679,-0.0664326,0.0602986,0.0137223,0.0854968,-0.0674329,0.0094739,-0.0127248,-0.0342066,0.0484068,-0.0353711,-0.130549,0.0712207,-0.123888,-0.100613,-0.0304281,-0.0378374,0.0204378,0.00616305,-0.0245083,-0.0399212,0.0224983,-0.04365,-0.275593,-0.119249,0.120046

In [69]:
from h2o.estimators import H2OGradientBoostingEstimator
gbm_plus_summary = H2OGradientBoostingEstimator()

gbm_plus_summary.train(y = "is_dad_joke", training_frame = ext_reviews)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [72]:
gbm_plus_summary.auc(valid = True)

In [90]:
def predict(reviews, w2v, gbm):
    
    words = tokenize(reviews["dad_joke"].ascharacter())
    reviews_vec = w2v.transform(words, aggregate_method="AVERAGE")
    
    model_data = reviews.cbind(reviews_vec)
    return gbm.predict(model_data)


In [105]:
dad_joke = h2o.H2OFrame([["How can you tell you have a really bad case of acne? It’s when the blind try to read your face."]])



dad_joke.col_names = ["dad_joke"]

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [106]:
predict_var = predict(dad_joke, w2v_model, gbm_plus_summary)


gbm prediction progress: |████████████████████████████████████████████████| 100%


In [107]:
x = predict_var.as_data_frame()

In [108]:
x['predict'][0]

0.28712110737112817